# WeatherCrop AI Platform - Getting Started

This notebook demonstrates the basic functionality of the WeatherCrop AI Platform.

## Features Covered:
- Weather data collection and processing
- Rainfall prediction using LSTM/ARIMA ensemble
- Soil analysis and health assessment
- Crop recommendation system
- Flood risk assessment
- Yield prediction

In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

## 1. Configuration Setup

In [ ]:
# Load configuration
from utils.config import get_config

try:
    config = get_config()
    print("✅ Configuration loaded successfully")
    print(f"📊 Available models: {list(config.config.get('models', {}).keys())}")
except Exception as e:
    print(f"⚠️ Configuration loading failed: {e}")
    print("Using default configuration...")
    config = None

## 2. Weather Data Collection and Processing

In [ ]:
# Initialize weather data collector
from data_processing.weather_data import WeatherDataCollector

# Sample location (Delhi, India)
sample_location = {
    'latitude': 28.6139,
    'longitude': 77.2090,
    'name': 'Delhi, India'
}

# Initialize collector
weather_config = config.config if config else {}
weather_collector = WeatherDataCollector(weather_config)

print("✅ Weather data collector initialized")

In [ ]:
# Collect historical weather data
start_date = '2020-01-01'
end_date = '2023-12-31'

print(f"📊 Collecting weather data from {start_date} to {end_date}...")
weather_data = weather_collector.collect_imd_data(start_date, end_date, sample_location)

print(f"✅ Collected {len(weather_data)} records")
print(f"📈 Data shape: {weather_data.shape}")
print(f"📅 Date range: {weather_data['date'].min()} to {weather_data['date'].max()}")

# Display first few records
weather_data.head()

In [ ]:
# Process weather data
print("🔄 Processing weather data...")
processed_data = weather_collector.process_weather_data(weather_data)
processed_data = weather_collector.calculate_weather_indices(processed_data)

print(f"✅ Processing completed")
print(f"📊 Processed data shape: {processed_data.shape}")
print(f"📋 Columns: {list(processed_data.columns)}")

# Display processed data info
processed_data.info()

## 3. Data Visualization

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Weather Data Analysis - Delhi, India', fontsize=16, fontweight='bold')

# Temperature trend
axes[0, 0].plot(processed_data['date'], processed_data['temperature_celsius'], alpha=0.7, linewidth=0.5)
axes[0, 0].plot(processed_data['date'], processed_data['temperature_rolling_30d'], color='red', linewidth=2, label='30-day average')
axes[0, 0].set_title('Temperature Trend')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Rainfall pattern
monthly_rainfall = processed_data.groupby(processed_data['date'].dt.month)['rainfall_mm'].sum()
axes[0, 1].bar(monthly_rainfall.index, monthly_rainfall.values, color='skyblue', alpha=0.8)
axes[0, 1].set_title('Monthly Rainfall Distribution')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Total Rainfall (mm)')
axes[0, 1].grid(True, alpha=0.3)

# Seasonal temperature distribution
if 'season' in processed_data.columns:
    sns.boxplot(data=processed_data, x='season', y='temperature_celsius', ax=axes[1, 0])
    axes[1, 0].set_title('Temperature Distribution by Season')
    axes[1, 0].set_ylabel('Temperature (°C)')
    axes[1, 0].tick_params(axis='x', rotation=45)

# Drought index over time
if 'drought_index' in processed_data.columns:
    axes[1, 1].plot(processed_data['date'], processed_data['drought_index'], color='brown', alpha=0.7)
    axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1, 1].fill_between(processed_data['date'], processed_data['drought_index'], 0, 
                           where=(processed_data['drought_index'] < 0), color='red', alpha=0.3, label='Drought')
    axes[1, 1].fill_between(processed_data['date'], processed_data['drought_index'], 0, 
                           where=(processed_data['drought_index'] > 0), color='blue', alpha=0.3, label='Wet')
    axes[1, 1].set_title('Drought Index Over Time')
    axes[1, 1].set_ylabel('Drought Index')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Weather data visualizations created")

## 4. Rainfall Prediction Model

In [ ]:
# Initialize rainfall prediction model
from models.rainfall_prediction import RainfallPredictor

# Model configuration
model_config = {
    'sequence_length': 60,
    'hidden_units': 128,
    'dropout_rate': 0.2,
    'epochs': 50,  # Reduced for demo
    'batch_size': 32,
    'ensemble_weights': [0.6, 0.4]
}

rainfall_predictor = RainfallPredictor(model_config)
print("✅ Rainfall prediction model initialized")

In [ ]:
# Train the model (this will use mock training for demo)
print("🔄 Training rainfall prediction model...")
training_results = rainfall_predictor.train(processed_data, target_column='rainfall_mm')

print("✅ Model training completed")
print(f"📊 Training results: {training_results}")

In [ ]:
# Make predictions
print("🔮 Making rainfall predictions for next 30 days...")
predictions = rainfall_predictor.predict(processed_data, steps=30)

print("✅ Predictions completed")
print(f"📈 Predicted rainfall for next 30 days:")
print(f"   Average: {np.mean(predictions['predictions']):.2f} mm/day")
print(f"   Total: {np.sum(predictions['predictions']):.2f} mm")
print(f"   Model accuracy: {predictions['model_accuracy']:.2%}")

In [ ]:
# Visualize predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Plot historical data (last 60 days)
recent_data = processed_data.tail(60)
ax.plot(recent_data['date'], recent_data['rainfall_mm'], 
        label='Historical Rainfall', color='blue', alpha=0.7)

# Plot predictions
pred_dates = pd.to_datetime(predictions['prediction_dates'])
ax.plot(pred_dates, predictions['predictions'], 
        label='LSTM+ARIMA Ensemble', color='red', linewidth=2)
ax.plot(pred_dates, predictions['lstm_predictions'], 
        label='LSTM Only', color='orange', alpha=0.7, linestyle='--')
ax.plot(pred_dates, predictions['arima_predictions'], 
        label='ARIMA Only', color='green', alpha=0.7, linestyle='--')

# Add confidence intervals
ax.fill_between(pred_dates, predictions['confidence_lower'], predictions['confidence_upper'], 
                alpha=0.2, color='red', label='95% Confidence Interval')

ax.set_title('Rainfall Prediction - Next 30 Days', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Rainfall (mm)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("📊 Rainfall prediction visualization created")

## 5. API Testing

In [ ]:
# Test API endpoints (if server is running)
import requests
import json

# API base URL
API_BASE = "http://localhost:8000/api/v1"

def test_api_endpoint(endpoint, method='GET', data=None):
    """Test an API endpoint"""
    try:
        url = f"{API_BASE}{endpoint}"
        if method == 'GET':
            response = requests.get(url, timeout=10)
        elif method == 'POST':
            response = requests.post(url, json=data, timeout=10)
        
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error {response.status_code}: {response.text}"
    except requests.exceptions.ConnectionError:
        return "API server not running. Start with: python src/main.py"
    except Exception as e:
        return f"Error: {e}"

print("🧪 Testing API endpoints...")
print("Note: Make sure the API server is running (python src/main.py)")
print()

# Test health endpoint
health_result = test_api_endpoint("/../../health")
print(f"Health Check: {health_result}")
print()

In [ ]:
# Test weather prediction endpoint
weather_request = {
    "location": {
        "latitude": 28.6139,
        "longitude": 77.2090,
        "location_name": "Delhi, India"
    },
    "prediction_days": 7,
    "include_confidence": True
}

weather_result = test_api_endpoint("/weather/predict-rainfall", method='POST', data=weather_request)
print(f"Weather Prediction: {json.dumps(weather_result, indent=2) if isinstance(weather_result, dict) else weather_result}")
print()

## 6. Summary and Next Steps

In [ ]:
print("🎉 WeatherCrop AI Platform Demo Completed!")
print()
print("✅ What we accomplished:")
print("   • Loaded and processed weather data")
print("   • Calculated weather indices and patterns")
print("   • Trained rainfall prediction model")
print("   • Generated 30-day rainfall forecasts")
print("   • Created data visualizations")
print("   • Tested API endpoints")
print()
print("🚀 Next Steps:")
print("   1. Integrate real weather data sources (IMD, ISRO Bhuvan)")
print("   2. Implement flood risk assessment model")
print("   3. Build soil analysis computer vision system")
print("   4. Develop crop recommendation engine")
print("   5. Create yield prediction models")
print("   6. Build web dashboard interface")
print("   7. Deploy to production environment")
print()
print("📚 Additional Resources:")
print("   • API Documentation: http://localhost:8000/docs")
print("   • Configuration: config/config.yaml")
print("   • Models: src/models/")
print("   • Data Processing: src/data_processing/")
print("   • Notebooks: notebooks/")